# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## LLMChain

 A simple LLM Chain that contains a single input and a single output with LLM accepting a single Prompt

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [17]:
llm = ChatOpenAI(temperature = 0.7, model = llm_model)

In [27]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a game which involves {game_description}?"
)

In [28]:
chain = LLMChain(llm=llm, prompt=prompt)

In [30]:
game_description = "Supercar Racing"
chain.invoke({"game_description": game_description})

{'game_description': 'Supercar Racing', 'text': '"Supercar Grand Prix"'}

## SimpleSequentialChain

This chain can contain multiple sub-chains that accept single input, then give out single output. 
 
The received output is then passed to the next chain as input.

<u>Note:</u> _Its usecase is to sequentially process different requests via multiple prompts._

In [24]:
from langchain.chains import SimpleSequentialChain

In [32]:
llm = ChatOpenAI(temperature = 0.7, model = llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
       "What is the best name to describe \
    a game which involves {game_description}?"
)

# Chain 1
chain_one = LLMChain(llm = llm, prompt = first_prompt)

In [33]:
#prompt_template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description about the following game: {game_name}"
)

# Chain 2
chain_two = LLMChain(llm = llm, prompt=second_prompt)

In [34]:
simple_chain = SimpleSequentialChain(chains = [chain_one, chain_two], verbose = True)

In [38]:
simple_chain.invoke({"input": game_description})



> Entering new SimpleSequentialChain chain...
"Supercar Grand Prix" would be a great name to describe a game that involves Supercar Racing.
Experience the ultimate thrill of Supercar Racing in "Supercar Grand Prix". Compete against the best drivers and dominate the track.

> Finished chain.


{'input': 'Supercar Racing',
 'output': 'Experience the ultimate thrill of Supercar Racing in "Supercar Grand Prix". Compete against the best drivers and dominate the track.'}

## SequentialChain

- A complex version of Simple Sequential Chain.
- Allows multiple tasks to be distributed across sub-chains, sequentially processed.
- Allows for multiple inputs any previous sub-chain (as compared to immediate neighbor chain in case of SimpleSequentialChain

In [72]:
from langchain.chains import SequentialChain

In [73]:
llm = ChatOpenAI(temperature = 0.7, llm = llm_model)

In [74]:
# Prompt Template 1: Translate the sentence to English
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to German:"
    "\n\n{Review}"
)

# Chain 1: input = Review and Output = German_Review
chain_one = LLMChain(llm = llm, prompt = first_prompt, output_key = "German_Review")

In [75]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence: "
    "\n\n{German_Review}"
)

# Chain 2: input = "German_Review" and Output = summary
chain_two = LLMChain(llm = llm, prompt = second_prompt, output_key="summary")

In [76]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review: \n\n {Review}"
)

# Chain 3: input = Review, Output = language
chain_three = LLMChain(llm = llm, prompt = third_prompt, output_key="language")

In [77]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\n Language: {language}"
)

# Chain 4: input = "summary, language", Output = "followup_message"
chain_four = LLMChain(llm = llm, prompt=fourth_prompt, output_key = "followup_message")

In [78]:
# Overll chain: input = Review
# Output = German_Review, summary, language, followup_message
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables = ["German_Review", "summary", "language", "followup_message"],
    verbose = True
)

In [79]:
overall_chain.invoke({"Review": """The SuperCar Grand Prix game was amazing. 
The graphics were fantastic. The game also supported rendering using Dedicated GPU which 
made the graphics quite realistic. Overall, a banger for the money."""})



> Entering new SequentialChain chain...


TypeError: create() got an unexpected keyword argument 'llm'

## Few more: RouterChain